In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho) #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto)#el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 1:alto, x in 1:ancho]
end

function moneda(N)
    ceil(Int64, round(rand(Uniform(-N-0.499, N+0.499))))
end

function paleta(mi_foto, paso, tamaño)
    aesthetics = load(mi_foto)
    alto, ancho =size(aesthetics)
    x0 = ceil(Int64, rand(Uniform(0,ancho)))
    #x0 = ceil(Int64, ancho/3)
    y0 = ceil(Int64, rand(Uniform(0,alto)))
    #y0 = ceil(Int64, alto/2)
    paleta = zeros(RGB{N0f8}, tamaño)
    for i = 1:length(paleta)
        paleta[i] = aesthetics[mod(y0,alto)+1,mod(x0,ancho)+1]
        x0 = x0 + moneda(paso)
        y0 = y0 + moneda(paso)
    end
    return paleta
end

# Un ejemplo de Anillos de Herman

Un **Anillo de Herman** es una componente periodica doblemente conexa del conjunto de Fatou en la que la dinámica es conjugada a una rotación irracional.

El ejemplo que veremos es la función $$ f_{\alpha} (z) = e^{2 \pi i \alpha} z^2 \frac{z - 4}{1 - 4 z}. $$
Ésta función preserva el circulo unitario y los puntos $\infty$ y 0 son fijos bajo $f_{\alpha}$.

La derivada de $f_{\alpha}$ es $$ f_{\alpha}' (z) =  - e^{2 \pi i \alpha} z \frac{8 z^2 - 19 z + 8}{(1 - 4 z)^2},$$ y sus puntos criticos son $0$, $\infty$ y $$ \frac{19 \pm \sqrt{105}}{16} \approx \lbrace 0.547, 1.828\rbrace.$$

Supongamos que el circulo unitario esta contenido en una componente del conjunto de Fatou distinta a la cuenca de atracción de 0 o de $\infty$. Necesariamente debe ser un anillo y esta componente debe ser invariante.

## El experimento

Hay un punto critico dentro del disco unitario y otro fuera (ademas de los super-atractores). Vamos a usar una trampa similar a la que usamos en los discos de siegel pero en forma de anillo. Requerimos de los radios interno y externo del anillo trampa. Suponiendo que los puntos criticos se acumulan en la frontera del hipotético anillo podemos aproximar radios ``r_int`` y ``r_ext``. Iterando suficientes veces el punto critico dentro del disco unitario podemos aproximar una cota inferior al radio interno usando el maximo de los modulos de la órbita observada. De manera similar, obtenemos una cota superior para el radio externo usando el minimo del modulo de la órbita observada.

In [ ]:
function trampa_herman(Z, λ, r_int, r_ext, colores, eps = 0.005, R = 20.0, TOPE = 1000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps
            return RGB{Float64}(1.0, 1.0, 1.0) #blanco atractor en cero
        elseif abs(Z) > R
            return RGB{Float64}(1.0, 1.0, 1.0) #blanco atractor en infinito
        elseif abs(Z) > r_int && abs(Z) < r_ext
            n = 1 + mod(cont, length(colores)) #trampa de anillo
            return colores[n]
        end
        Z = λ*Z*Z*(Z - 4.0)/(1.0 - 4.0*Z)
        cont += 1
    end
    return colores[length(colores)]
end

function ejemplo_herman(α, paleta, eps=0.005, R=20.0, TOPE=1000, radio=5.0, centro=0.0, alto=1000, ancho=1000)
    λ = cos(2.0*pi*α) + sin(2.0*pi*α)*1.0im
    c_int = (19.0 - sqrt(105.0))/16.0
    c_ext = (19.0 + sqrt(105.0))/16.0
    r_int = eps
    r_ext = R
    for i = 0:TOPE
        r_int < abs(c_int) ? r_int = abs(c_int) : nothing
        r_ext > abs(c_ext) ? r_ext = abs(c_ext) : nothing
        c_int = λ*c_int*c_int*(c_int - 4.0)/(1.0 - 4.0*c_int)
        c_ext = λ*c_ext*c_ext*(c_ext - 4.0)/(1.0 - 4.0*c_ext)
    end
    broadcast(Z -> trampa_herman(Z, λ, r_int, r_ext, paleta, eps, R, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
mi_paleta = paleta("aesthetic.jpg", 10, 1000)

In [ ]:
pic = ejemplo_herman(0.5*(1.0 + sqrt(5.0)), mi_paleta, 0.05, 10.0, 1000, 5.5, 1.6, 1000, 1000)